In [33]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender, bm25_weight
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix

In [2]:
data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K"
dest_data_dir = "/Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest"

In [3]:
def load_data(path): 
    return pd.read_table(
        os.path.join(path, "usersha1-artmbid-artname-plays.tsv"),
        usecols=[0, 1, 2, 3],
        names=['user', 'artist', 'artist_name', 'plays'],
    )

def prepare_data(data):
    """Возвращает подготовленные данные и словарь артистов"""
    # Заменяем отсутствующие значения идентификаторов исполнителя на его имя
    print("Filling na artists...")
    data.dropna(subset=['artist_name'], inplace=True)
    empty_artist = data.artist.isnull()
    data.loc[empty_artist, 'artist'] = data.loc[empty_artist, 'artist_name']
    
    # Преобразуем пользователей и исполнителей в числа
    print("Encoding artits and users...")
    data['artist_id'] = data.artist.astype("category").cat.codes.copy() + 1
    data['user_id'] = data.user.astype("category").cat.codes.copy() + 1
    
    # Составляем словарь исполнителей
    print("Saving atists dict...")
    artists = dict(data.groupby('artist_id').artist_name.first())
    return data[['user_id', 'artist_id', 'plays']], artists

In [4]:
def split_data(data, size=0.2):
    data_train, data_test = train_test_split(data, test_size=size)
    test_user_set = set(data_test.user_id.unique())
    train_user_set = set(data_train.user_id.unique())
    # Оставляем только пользователей которые есть одновременно в тестовой и обучающей выборке
    user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
    return (
        data_train[~data_train.user_id.isin(user_ids_to_exclude)], 
        data_test[~data_test.user_id.isin(user_ids_to_exclude)]
    )

In [5]:
def save_data(path, name, data, columns=["user_id", "artist_id", "plays"]):
    file_name = os.path.join(path, name + ".tsv")
    data[columns].to_csv(file_name, sep="\t", header=False, index=False)

In [6]:
def make_coo_matrix(data):
    """Создаёт разреженную матрицу item*user"""
    return coo_matrix((
        data.plays.astype(np.double),
        (data.artist_id, data.user_id)
    ))

def sparse_info(sparse_matrix):
    """функция, которая красиво печатает информацию о разреженных матрицах"""
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}".format(
        sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1]
    ))
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [7]:
def make_user_recommendations(model, data_train_coo, data_test, path, model_name):
    """Генерирует рекомендации для каждого пользователя и сохраняет их в файл для проверки mrec-ом"""
    user_plays = data_train_coo.T.tocsr()
    file_name = "test.{}.tsv.recs.tsv".format(model_name)
    # Получаем столько рекомендаций, сколько записей у пользователя в тестовой выборке, но не меньше 3
    user_rec_counts = dict(data_test.user_id.value_counts())
    user_rec_counts = {user_id: max(10, int(N)) for user_id, N in user_rec_counts.items()}
    with open(os.path.join(path, file_name), "w") as output_file:
        for user_id in tqdm_notebook(data_test.user_id.unique()):
            for artist_id, score in model.recommend(user_id, user_plays, N=user_rec_counts[user_id]):
                output_file.write("{}\t{}\t{}\n".format(user_id, artist_id, score))    

In [8]:
def search_artist(artists, name_prefix):
    """Поиск по базе исполнителей: выводит список всех начинающихся с переданного префикса"""
    for artist_id, artist_name in artists.items():
        if artist_name.lower().startswith(name_prefix):
            print("[{}]\t{}".format(artist_id, artist_name))
            
def find_similar_artists(model, artist_id, artists, count=10):
    """Выводит список похожих исполнителей"""
    print("Similar to {}:".format(artists[artist_id]))
    print("-------------------------------")
    for similar_artist_id, score in model.similar_items(artist_id, count):
        print("{:.3f}\t{}".format(score, artists[similar_artist_id]))

In [9]:
data_src = load_data(data_dir)

In [10]:
data, artists = prepare_data(data_src.copy())

Filling na artists...
Encoding artits and users...
Saving atists dict...


In [11]:
data.head()

,user_id,artist_id,plays
0,1,39710,2137
1,1,211105,1099
2,1,135574,897
3,1,40727,717
4,1,143198,706


In [12]:
data_train, data_test = split_data(data, 0.2)
data_train.shape, data_test.shape

((14025188, 3), (3507072, 3))

In [13]:
data_train.head()

,user_id,artist_id,plays
6300863,128940,5546,276
9157993,187350,165235,37
6457294,132137,94933,291
5271168,107872,184815,48
7087033,144995,178286,48


In [14]:
save_data(dest_data_dir, 'train', data_train)

In [15]:
data_train_coo = make_coo_matrix(data_train)

In [16]:
sparse_info(data_train_coo)

Размерности матрицы: (268590, 358869)
Ненулевых элементов в матрице: 14025188
Доля ненулевых элементов: 0.00014550667199783102
Среднее значение ненулевых элементов: 215.0617464806889
Максимальное значение ненулевых элементов: 419157.0
Минимальное значение ненулевых элементов: 1.0


In [17]:
data_train_csr = data_train_coo.tocsr()

In [18]:
# строим матрицу схожести по косинусной мере
models = {}
models['knn_cosine'] = CosineRecommender()
models['knn_cosine'].fit(data_train_csr)

In [19]:
save_data(dest_data_dir, 'test.knn_cosine', data_test)

In [20]:
# Попробуем найти похожих исполнителей на Aphex Twin, чтобы убедиться что рекомендации вообще работают
search_artist(artists, 'aphex twin')

[128602]	aphex twin & céline eia
[128603]	aphex twin & m-ziq (aka rich & mike)
[128604]	aphex twin & squarepusher
[128605]	aphex twin & µ-ziq (mike & rich)
[128606]	aphex twin & µ-ziq - mike and rich
[128607]	aphex twin (aka afx)
[128608]	aphex twin (aka caustic window)
[128609]	aphex twin (aka polygon window)
[128610]	aphex twin (aka universal indicator)
[128611]	aphex twin - power-pill
[128612]	aphex twin ft. photek & autechre
[128613]	aphex twin vs luke vibert
[128614]	aphex twin vs run jeremy
[128615]	aphex twin/afx
[128616]	aphex twin/luke vibert
[128617]	aphex twin/richard james
[210592]	aphex twin


In [22]:
aphex_twin_id = 210592
find_similar_artists(models['knn_cosine'], aphex_twin_id, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.436	afx
0.354	ケンイシイ
0.353	magnat
0.284	squarepusher
0.272	polygon window
0.266	autechre
0.213	boards of canada
0.205	gts feat. melodie sexton
0.198	apollon / muslimgauze


In [23]:
# Создаём рекомендации для всех тестовых пользователей
make_user_recommendations(models['knn_cosine'], data_train_csr, data_test, dest_data_dir, 'knn_cosine')

HBox(children=(IntProgress(value=0, max=358538), HTML(value='')))

In [24]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.knn_cosine.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

[2017-12-05 01:15:40,695] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.knn_cosine.tsv...
None
mrr            0.1176 +/- 0.0000
prec@5         0.0427 +/- 0.0000
prec@10        0.0348 +/- 0.0000
prec@15        0.0254 +/- 0.0000
prec@20        0.0192 +/- 0.0000


In [47]:
# Тренируем ALS-модель
data_train_bm25 = bm25_weight(data_train_coo, K1=100, B=0.8).tocsr()
models['als'] = AlternatingLeastSquares(factors=128, iterations=20, regularization=0.001)
models['als'].fit(data_train_bm25)

In [48]:
find_similar_artists(models['als'], aphex_twin_id, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.996	boards of canada
0.992	squarepusher
0.986	amon tobin
0.981	autechre
0.980	dj shadow
0.980	burial
0.980	kraftwerk
0.979	björk
0.978	portishead


In [49]:
save_data(dest_data_dir, 'test.als', data_test)

In [50]:
make_user_recommendations(models['als'], data_train_bm25, data_test, dest_data_dir, 'als')

HBox(children=(IntProgress(value=0, max=358538), HTML(value='')))

In [51]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

[2017-12-05 04:40:37,789] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als.tsv...
None
mrr            0.3166 +/- 0.0000
prec@5         0.1392 +/- 0.0000
prec@10        0.1110 +/- 0.0000
prec@15        0.0804 +/- 0.0000
prec@20        0.0608 +/- 0.0000


In [52]:
from implicit.approximate_als import NMSLibAlternatingLeastSquares

In [53]:
models['als_nmslib'] = NMSLibAlternatingLeastSquares(factors=128)
models['als_nmslib'].fit(data_train_bm25)

In [54]:
find_similar_artists(models['als_nmslib'], aphex_twin_id, artists)

Similar to aphex twin:
-------------------------------
1.000	aphex twin
0.996	boards of canada
0.992	autechre
0.986	amon tobin
0.983	dj shadow
0.981	squarepusher
0.981	burial
0.979	massive attack
0.979	björk
0.978	orbital


In [55]:
make_user_recommendations(models['als_nmslib'], data_train_bm25, data_test, dest_data_dir, 'als_nmslib')

HBox(children=(IntProgress(value=0, max=358538), HTML(value='')))

In [56]:
save_data(dest_data_dir, 'test.als_nmslib', data_test)

In [57]:
!/Users/ur001/.pyenv/versions/netology1/bin/mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als_nmslib.tsv \
    --recsdir /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest

[2017-12-05 04:48:42,178] INFO: processing /Users/ur001/Documents/Datasets/lastfm-dataset-360K/dest/test.als_nmslib.tsv...
None
mrr            0.2426 +/- 0.0000
prec@5         0.1043 +/- 0.0000
prec@10        0.0818 +/- 0.0000
prec@15        0.0590 +/- 0.0000
prec@20        0.0446 +/- 0.0000
